In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
from plotly.offline import iplot

In [ ]:
# Lecture des données d'apprentissage et de test
df1 = pd.read_csv("../input/student-alcohol-consumption/student-por.csv")
df2 = pd.read_csv("../input/student-alcohol-consumption/student-mat.csv")

In [ ]:
df1.head()

In [ ]:
df1.count()

On peut voir que l'ensemble des données sont remplies pour chacun des étudients

In [ ]:
hommes = (df1.sex=="M")
femmes = (df1.sex=="F")

In [ ]:
df1[hommes].head()  

In [ ]:
fig = sns.FacetGrid(df1, hue="romantic", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "G3", shade=True)
fig.add_legend()

On peut voir que le fait d'être en couple à un léger impact sur le résultat final des élèves, les élèves en couples ont des réultat un peu moins bons que les autres.


In [ ]:
fig = sns.FacetGrid(df1, hue="Walc", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "traveltime", shade=True)
fig.add_legend()

Je pensais dans un premier temps que le temps de trajet avait un impact sur la fréquence de consommation d'alcool mais non. Je pensais qu'il aurait était plus difficile pour les personnes habitant loins de pouvoir boire et reprendre la route par la suite.

In [ ]:
fig = sns.FacetGrid(df1, hue="Walc", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "absences", shade=True)
fig.add_legend()

On peut voir sur ce graphique que la consommation d'alcool joue un petit role dans le nombre d'abscence des eleves ( je pense notamment à la courbe verte s'applatissant moins rapidement que les autres)

In [ ]:
df1[df1["school"] == 'GP']['reason'].value_counts().plot(kind='bar')

In [ ]:
labels = df1["Fjob"].unique().tolist()
amount = df1["Fjob"].value_counts().tolist()

colors = ["orange", "green", "yellow", "white",'cyan']

trace = go.Pie(labels=labels, values=amount,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

dt = [trace]
layout = go.Layout(title="Mother's job")

fig = go.Figure(data=dt, layout=layout)
iplot(fig, filename = 'pi_chart')

On peut remarquer que beaucoup d'etudiants des ces écoles ont pour père un professeur.

Il est necessaire pour traité les données dans un randomForest de les formaté en entier

In [ ]:
# creating a dict file  
gender = {'M': 1,'F': 2} 
yesno = {'yes': 1, 'no' : 0}
schooltoint = {'GP': 1, 'MS' : 2}
addresstoint = {'U' : 1, 'R' : 2}
famsizetoint = {'GT3' : 1, 'LE3' : 2}
Pstatustoint = {'T' : 1, 'A' : 2}
  
# traversing through dataframe 
# Gender column and writing 
# values where key matches 
df1.sex = [gender[item] for item in df1.sex] 
df1.schoolsup = [yesno[item] for item in df1.schoolsup] 
df1.famsup = [yesno[item] for item in df1.famsup] 
df1.paid = [yesno[item] for item in df1.paid] 
df1.activities = [yesno[item] for item in df1.activities] 
df1.nursery = [yesno[item] for item in df1.nursery] 
df1.higher = [yesno[item] for item in df1.higher] 
df1.internet = [yesno[item] for item in df1.internet] 
df1.romantic = [yesno[item] for item in df1.romantic]
df1.school = [schooltoint[item] for item in df1.school]
df1.address = [addresstoint[item] for item in df1.address]
df1.famsize = [famsizetoint[item] for item in df1.famsize]
df1.Pstatus = [Pstatustoint[item] for item in df1.Pstatus]
df1 = df1.drop(['Mjob','Fjob','reason','guardian','Dalc'], axis=1)


df1.head() 

In [ ]:
df2.sex = [gender[item] for item in df2.sex] 
df2.schoolsup = [yesno[item] for item in df2.schoolsup] 
df2.famsup = [yesno[item] for item in df2.famsup] 
df2.paid = [yesno[item] for item in df2.paid] 
df2.activities = [yesno[item] for item in df2.activities] 
df2.nursery = [yesno[item] for item in df2.nursery] 
df2.higher = [yesno[item] for item in df2.higher] 
df2.internet = [yesno[item] for item in df2.internet] 
df2.romantic = [yesno[item] for item in df2.romantic]
df2.school = [schooltoint[item] for item in df2.school]
df2.address = [addresstoint[item] for item in df2.address]
df2.famsize = [famsizetoint[item] for item in df2.famsize]
df2.Pstatus = [Pstatustoint[item] for item in df2.Pstatus]
df2 = df2.drop(['Mjob','Fjob','reason','guardian','Dalc'], axis=1)


df2.head() 

In [ ]:
data_train = df1       # 80% des données avec frac=0.8
data_test = df2

In [ ]:
X_train = data_train.drop(['Walc'], axis=1)
y_train = data_train['Walc']
X_test = data_test.drop(['Walc'], axis=1)
y_test = data_test['Walc']

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

le score final de 0.81 n'est pas vraiment très bon, cela est dû au manque de données des tables

In [ ]:
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)

On peut voir que l'algorithme arrive mieux a prédire les personnes buvant souvant mais est moins bon pour prédire les personnes ne buvant pas beaucoup.
Il a donc plus tendances a dire que les perosnnes boivent alors que non (Fauxpositif)

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(12,8))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), df1.columns[indices])
plt.title('Importance des caracteristiques')

On peut remarquer que G1 est deuxième suivi de pres par G2 on peut en conclure que les personnes ayant réussi leur première période ou deuxième période de cour ont plus tendance a vouloir sortir (surement car ils se disent qu'il ont bien travaillés et qu'il n'ont pas de difficulté particulière). A l'inverse les personnes n'ayant pas vraiment réussi les premières période boivent moins, surement dans le but de consacrer plus de temps au travail.
En toute logique les personnes qui sortent le plus sont celles qui boivent le plus.
On peut aussi étonament retrouver que le facteur de la santé de l'étudiant a un impact important sur sa fréquence de consommation d'alcool.

In [ ]:
df2 = df2.drop(['traveltime'], axis=1)
df1 = df1.drop(['traveltime'], axis=1)

Je supprime des données de la table cette info car non pertinante pour la recherche qui va suivre

In [ ]:
data_train = df1      
data_test = df2

In [ ]:
X_train = data_train.drop(['sex'], axis=1)
y_train = data_train['sex']
X_test = data_test.drop(['sex'], axis=1)
y_test = data_test['sex']

In [ ]:
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)

En reposant sur les données de la table l'algorithme est capable de prédire a 90 son sexe, il est surement imparfait encore une fois dû au manque de donées.

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(12,8))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), df1.columns[indices])
plt.title('Importance des caracteristiques')

On peut retrouver ici les caractèristiques majeurs qui différencies les hommes des femmes selon l'algorithme et ses données en entrée.
On peut voir que la caractéristiques principal qui les différencis sont la fréquence de sortie, suivie par le nombre d'absence.
Dans un premier temps j'avais laissé le temps de trajet dans le tableau de données, étant une donnée apparaissant comme critère principal je l'ai supprimé car elle n'avait pas vraiment de sens.